# Packages 

In [3]:
import pandas as pd
import numpy as np
from scipy import optimize
from datetime import date
from datetime import datetime
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import os
import json
import io
import base64
import re
import copy

import dash
from dash.exceptions import PreventUpdate
from dash import dcc
from dash import html
from dash import callback_context
from dash import dash_table
import plotly.express as px
from dash.dependencies import Output,Input,State
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots


# Images


In [4]:
#Path for RSM logo
cwd=os.getcwd()

fpath='.\images\initial.jpg'

test_base64 = base64.b64encode(open(fpath, 'rb').read()).decode('ascii')

# App layout and controls

In [5]:
app=dash.Dash(__name__,external_stylesheets=[dbc.themes.SPACELAB,
                                             "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css"])

# df=pd.read_csv('.\data\AutoBio_SelectedTeam.csv')

# test=dbConnect('accredible_cred_descriptions')


app.layout=html.Div([
    
#RSM colors and app title
    dbc.Row([dbc.Col(
                    html.H1('Anger',style={"color":"black","background-color":"#F13027",
                                          "text-align":"center"})
                    ,width={'size':2,'offset':0,'order':1}
                    ),
            dbc.Col(
                    html.H1("Love",style={"color":"black","background-color":"#F127A1",
                                          "text-align":"center"})
                    ,width={'size':2,'order':2}
                    ),
            dbc.Col(
                    html.H1("Joy",style={"color":"black","background-color":"#FFF633",
                                        "text-align":"center"})
                    ,width={'size':2,'order':3}
                    ), 
            dbc.Col(
                    html.H1("Surprise",style={"color":"black","background-color":"#27F143",
                                             "text-align":"center"})
                    ,width={'size':2,'order':4}
                    ), 
            dbc.Col(
                    html.H1("Sadness",style={"color":"black","background-color":"#279BF1",
                                            "text-align":"center"})
                    ,width={'size':2,'order':5}
                    ), 
            dbc.Col(
                    html.H1("Fear",style={"color":"black","background-color":"#9827F1",
                                         "text-align":"center"})
                    ,width={'size':2,'order':6}
                    ), 
            ]),

#LOB, Region, and Level Dropdown Selectors
    dbc.Row([dbc.Col([
                html.Div(["Emotion"],style={'font-size':24}),
                dcc.Dropdown(id="emotion",options=[{'label':'Joy','value':'joy'},
                                                   {'label':'Anger','value':'anger'},
                                                   {'label':'Sadness','value':'sadness'},
                                                   {'label':'Love','value':'love'},
                                                   {'label':'Surprise','value':'surprise'},
                                                   {'label':'Fear','value':'fear'},
                                                  ],
                             placeholder='-Select Emotion-',clearable=True,
                             style={'font-size':24,
                                   'vertical-align':'text-bottom',
                                   'padding-bottom':'5px'}
                            )
                    ],width={'size':1,'offset':1}),#End dbc.Col
             
    ]),#End dbc.Row
    
#Free text input for user text.  The dcc store is used to store data in browser and
    #can share data between callbacks 
    dbc.Row([dbc.Col([
                    html.Div(["How are you feeling"],style={'font-size':24}),
                    dcc.Input(id='input',placeholder='Enter text..',type='text',style={'font-size':24})],
            width={'size':6,'offset':1}),
            ]),

    dcc.Download(id='download_excel'),    
    dcc.Store(id='saved_data',data={}),
    dcc.Store(id='df_filtered',data={}),

#Download team button and RSM image
    dbc.Row([
        dbc.Col([html.Br(),
                 html.Button(id='save_excel',n_clicks=0,
                             style={"color":"white","background-color":"#3F9C35",'font-size':24,
                                  'border-radius':'8px'},
                             children=[html.I(className="fa fa-download mr-1"),"Generate team"])],
                width={'offset':1,'order':1}),

        dbc.Col(html.Img(id='picture',src='data:image/png;base64,{}'.format(test_base64)),
                width={'offset':4,'order':2}),

            ]),#End dbc.Row
    
#Engagement Team Options Checkboxes
    dbc.Row([dbc.Col([
                html.Br(),
                html.Div(["Engagement Team Options: "],style={'font-size':24}),
                dcc.Checklist(id='checkList_pop',
                             options=[],
                             value=[],
                             style={'font-size':24,'border':'2px #009DCE solid'},
                             inputStyle={'cursor':'pointer'},
                             labelStyle={'background':'white','border-radius':'0.5rem',
                                         'display':'block',},
                             )],
            width={'offset':1}), #End dbc.Col for checkboxes
             
             dbc.Col([
                 dash_table.DataTable(
                     id="table",
#                      data={},
#                      columns=[]
#                      data=df.to_dict('records'),
#                      columns=[{"name": i, "id": i} for i in df.columns if i not in \
#                              ['First Name','Last Name','Location City','Location State/Province']]
                 ) #end DataTable
                 
             ],width={'size':7}) #End dbc.Col fordash data table 
        
            ]), #End dbc.Row

                    
]) #End html.Div

        

# Callbacks


In [6]:
#Populate the image based on value of emotion checkbox

@app.callback(
    Output('picture', 'src'),
    Input(component_id='emotion',component_property='value')
            )
    
def set_image(chosen_emotion):
    
    if chosen_emotion is None:
        raise PreventUpdate
    
    fpath=f".\images\{chosen_emotion}.jpg"
    
    test_base64 = base64.b64encode(open(fpath, 'rb').read()).decode('ascii')
    src='data:image/png;base64,{}'.format(test_base64)
    
#     if chosen_lob in ['Audit','Consulting','Tax','ICS']:
#         options=[{'label': c, 'value': c} for c in sorted(df_filtered['Full Name'].unique())]
        
        #         options=[{'label':'Audit Team Member A','value':'Audit Team Member A'},
#                  {'label':'Audit Team Member B','value':'Audit Team Member B'}]


    
    return src

In [7]:
#Populate the data and the columns of the dash_table

@app.callback(
    Output('table','data'),
    Output('df_filtered','data'),
    Input(component_id='lob',component_property='value')
            )


def set_table_data_cols(chosen_lob):
    
    if chosen_lob is None:
        raise PreventUpdate
    
    if chosen_lob in ['Audit','Consulting','Tax','ICS']:
        
        #The key to getting this to work was changing df to df_filtered (i.e another name)
        df_filtered=df.loc[df['LOB']==chosen_lob].copy()
        
        data_table_data=df_filtered.to_dict('records')
    

    else:
        data_table_data={}

    return data_table_data,data_table_data

In [8]:
#Download team list
@app.callback(
    Output(component_id='download_excel',component_property='data'),
    Input(component_id='save_excel',component_property='n_clicks'),
    State(component_id='saved_data',component_property='data'),
    State(component_id='df_filtered',component_property='data'),
    prevent_initial_call=True,
            )

def download_excel(save_excel_button_clicks,values,df_filtered):
    
    print(f'Number of button clicks {save_excel_button_clicks}')
    
    if save_excel_button_clicks==0:
        raise PreventUpdate
    
    else:
        
#         final_df=pd.DataFrame(values,columns=['Team Members'])

        #May want to change this setup so the df_filtered was already filtered (like values was in get_teams_list)
        final_df_test=pd.DataFrame(df_filtered)
        
        final_df=final_df_test.loc[final_df_test['Full Name'].isin(values)]

        
        curr_date=datetime.now().date()
        
        
    return dcc.send_data_frame(final_df.to_excel, f'{curr_date}_results.xlsx', sheet_name='Results',index=False)

In [ ]:
if __name__=='__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
